# CI/CD for a Kubeflow pipeline on Vertex AI

In this lab you will walk through authoring of a **Cloud Build** CI/CD workflow that automatically builds, deploys, and runs a Kubeflow pipeline on Vertex AI. You will also integrate your workflow with **GitHub** by setting up a trigger that starts the  workflow when a new tag is applied to the **GitHub** repo hosting the pipeline's code.

## Configuring environment settings

In [69]:
PROJECT_ID = !(gcloud config get-value project)
PROJECT_ID = PROJECT_ID[0]
REGION = "us-east1"
ARTIFACT_STORE = f"gs://{PROJECT_ID}-beer-artifact-store"

Let us make sure that the artifact store exists:

In [70]:
!gsutil ls | grep ^{ARTIFACT_STORE}/$ || gsutil mb -l {REGION} {ARTIFACT_STORE}

gs://qwiklabs-asl-04-5e165f533cac-beer-artifact-store/


## Creating the KFP CLI builder for Vertex AI
### Review the Dockerfile describing the KFP CLI builder

In [83]:
%%writefile kfp-cli_vertex/Dockerfile

FROM gcr.io/deeplearning-platform-release/base-cpu
RUN pip install kfp==1.6.6
RUN pip install google-cloud-aiplatform==1.3.0
RUN pip install -U google-cloud-pipeline-components==0.2.1

ENTRYPOINT ["/bin/bash"]

Overwriting kfp-cli_vertex/Dockerfile


### Build the image and push it to your project's **Container Registry**.

In [84]:
KFP_CLI_IMAGE_NAME = "kfp-cli-vertex"
KFP_CLI_IMAGE_URI = f"gcr.io/{PROJECT_ID}/{KFP_CLI_IMAGE_NAME}:latest"
KFP_CLI_IMAGE_URI

'gcr.io/qwiklabs-asl-04-5e165f533cac/kfp-cli-vertex:latest'

In [ ]:
!gcloud builds submit --timeout 15m --tag {KFP_CLI_IMAGE_URI} kfp-cli_vertex

Creating temporary tarball archive of 4 file(s) totalling 1.4 KiB before compression.
Uploading tarball of [kfp-cli_vertex] to [gs://qwiklabs-asl-04-5e165f533cac_cloudbuild/source/1654849811.754844-d56258bb83804c6f8e0c0aaac498f8fb.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/qwiklabs-asl-04-5e165f533cac/locations/global/builds/2c3f9669-ac5f-4978-bfb3-287b974e6b5a].
Logs are available at [https://console.cloud.google.com/cloud-build/builds/2c3f9669-ac5f-4978-bfb3-287b974e6b5a?project=547029906128].
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "2c3f9669-ac5f-4978-bfb3-287b974e6b5a"

FETCHSOURCE
Fetching storage object: gs://qwiklabs-asl-04-5e165f533cac_cloudbuild/source/1654849811.754844-d56258bb83804c6f8e0c0aaac498f8fb.tgz#1654849812059196
Copying gs://qwiklabs-asl-04-5e165f533cac_cloudbuild/source/1654849811.754844-d56258bb83804c6f8e0c0aaac498f8fb.tgz#1654849812059196...
/ [1 files][  1.1 KiB/  1.1 KiB]                  

In [ ]:
%%writefile cloudbuild_vertex.yaml
# Copyright 2021 Google LLC

# Licensed under the Apache License, Version 2.0 (the "License"); you may not use this
# file except in compliance with the License. You may obtain a copy of the License at

# https://www.apache.org/licenses/LICENSE-2.0
    
# Unless required by applicable law or agreed to in writing, software 
# distributed under the License is distributed on an "AS IS"
# BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either 
# express or implied. See the License for the specific language governing 
# permissions and limitations under the License.

steps:
# Build the trainer image
- name: 'gcr.io/cloud-builders/docker'
  args: ['build', '-t', 'gcr.io/$PROJECT_ID/trainer_image_beer_vertex:latest', '.']
  dir: $_PIPELINE_FOLDER

# Compile the pipeline
- name: 'gcr.io/$PROJECT_ID/kfp-cli-vertex'
  args:
  - '-c'
  - |
    dsl-compile-v2 --py pipeline_prebuilt.py --output beer_kfp_pipline.json
  env:
  - 'PIPELINE_ROOT=gs://$PROJECT_ID-kfp-artifact-store/pipeline'
  - 'PROJECT_ID=$PROJECT_ID'
  - 'REGION=$_REGION'
  - 'SERVING_CONTAINER_IMAGE_URI=us-docker.pkg.dev/vertex-ai/prediction/sklearn-cpu.0-20:latest'
  - 'TRAINING_CONTAINER_IMAGE_URI=gcr.io/$PROJECT_ID/trainer_image_beer_vertex:latest'
  - 'TRAINING_FILE_PATH=gs://$PROJECT_ID-kfp-artifact-store/data/train.parquet'
  - 'VALIDATION_FILE_PATH=gs://$PROJECT_ID-kfp-artifact-store/data/valid.parquet'
  dir: pipeline
  
# Run the pipeline
- name: 'gcr.io/$PROJECT_ID/kfp-cli-vertex'
  args:
  - '-c'
  - |
    python kfp-cli_vertex/run_pipeline.py \
    --project_id=$PROJECT_ID \
    --template_path=./pipeline/beer_kfp_pipeline.json \
    --display_name=beer_kfp_pipeline \
    --region=$_REGION
    
# Push the images to Container Registry
# TODO: List the images to be pushed to the project Docker registry
images: ['gcr.io/$PROJECT_ID/trainer_image_beer_vertex:latest']


# This is required since the pipeline run overflows the default timeout
timeout: 10800s


## Manually triggering CI/CD runs

You can manually trigger **Cloud Build** runs using the [gcloud builds submit command]( https://cloud.google.com/sdk/gcloud/reference/builds/submit).

In [ ]:
SUBSTITUTIONS = f"_REGION={REGION},_PIPELINE_FOLDER=./training_app"
SUBSTITUTIONS

In [ ]:
!gcloud builds submit . --config cloudbuild_vertex.yaml --substitutions {SUBSTITUTIONS}